### Dostępność modelu środowiska

Metody oparte na **value iteration** czy **policy iteration** opierają się na wykorzystywaniu prawdopodobieństw przejść środowiska, aby wyznaczyć optymalne strategie.
W praktyce bardzo rzadko mamy wiedzę o wewnętrznej dynamice środowiska lub jest ona bardzo trudna do wyznaczenia.

Dużo prostszym zadaniem jest próbkowanie epizodów ze środowiska oraz użycie ich do wyznaczenia strategii bliskich optymalnym. Rodziną metod, które opierają się na tej idei, są metody Monte Carlo.


## Monte Carlo - model-free method

We can estimate the state values and action values in an MDP from random samples.
Monte Carlo (MC) estimation is a general concept that refers to making estimations through repeated random sampling. In the context of RL, it refers to
**a collection of methods that estimates state values and action values using sample trajectories of complete episodes**.
Using random samples is essential because often environment dynamics:
- Is too complex to deal with
- It is not known in the first place

Summing up:
- MC methods learn directly from episodes of experience
- MC is *model-free*: no knowledge of MDP transitions/rewards
- MC learns from *complete episodes*
- MC uses the simplest idea: value = mean return
- Caveat: can only apply MC to episodic MDPs (all episodes must terminate)

## Monte Carlo prediction

We need to be able to evaluate a given policy to be able to improve it. MC prediction suggests simply observing (many) sample trajectories, sequences of state-action-reward tuples, starting in $S$, to estimate expectation $v_\pi(s) = E_{\pi}[G_{t}|S_{t}=s]$.
MV policy evaluation uses *empirical mean* return instead of *expected* return.

#### Przykład

Robot umieszczony w środowisku typu *Grid World* otrzymuje nagrodę +1 za każdy ruch. Epizod kończy się, gdy robot wpadnie na ścianę. Strategia działania jest deterministyczna, lecz po wybraniu akcji, agent ma 70% szansy na poruszenie się w wybranym kierunku oraz po 10% szansy na każdy z pozostałych kierunków (dynamika środowiska).

Rysunek poniżej przedstawia deterministyczną strategię $\pi$ oraz dwie przykładowe trajektorie $\tau$ przejścia agenta przez środowisko (start w (1,2), epizod kończy się po wpadnięciu na ścianę)

![](img/example_robot.png)

Aby wyznaczyć wartość oczekiwaną wartości $V_{\pi}$ potrzebujemy (zgodnie z [definicją](https://pl.wikipedia.org/wiki/Warto%C5%9B%C4%87_oczekiwana)): sumy nagród z epizodu (*return*, $G_{\tau}$) oraz prawdopodobieństwa wystąpienia danego epizodu ($p_{\pi}(\tau)$)

#### $\tau_{1}$

- Kolejne stany: $S_{0}=(1,2), S_{1}=(1,1), S_{2}=(0,1), S_{3}=zderzenie$
- Kolejne akcje: $A_{0}=dół, A_{1}=góra, A_{2}=prawo$
- Kolejne nagrody: $R_{1}=1, R_{2}=1, R_{3}=0$
- $p_{\pi}(\tau_{1})=0.7*0.1*0.1=0.007$
- $G_{\tau_{1}}=1+\gamma 1+\gamma^{2}0$
-
#### $\tau_{2}$

- Kolejne stany: $S_{0}=(1,2), S_{1}=(2,2), S_{2}=(2,1), S_{3}=(3,1), S_{4}=(2,1), S_{5}=(2,0), S_{6}=zderzenie$
- Kolejne akcje: $A_{0}=dół, A_{1}=góra, A_{2}=prawo$
- Kolejne nagrody: $R_{1}=1, R_{2}=1, R_{3}=0$
- $p_{\pi}(\tau_{2})=0.1*0.7*0.1*0.7*0.1*0.1=0.000049$
- $G_{\tau_{2}}=1+\gamma1+\gamma^{2}1+\gamma^{3}1+\gamma^{4}1+\gamma^{5}0$


Aby wyznaczyć wartość stanu $s$, musimy wyznaczyć $v_{\pi}=E_{\pi}[G_{t}|S_{t}=s]=E_{\pi}[G_{\tau}|S_{\tau}=s]=\sum_{i}p_{\pi}(\tau_{i})*G_{\tau_{i}}$
Do powyższej formuły aplikujemy wszystkie epizody, które **rozpoczęły się w stanie $s$**.

Wyznaczenie jej jest bardzo trudne, a nawet niemożliwe z uwagi na potencjalnie nieskończoną liczbę możliwych epizodów. Dlatego zamiast wartości teoretycznej $v_{\pi}(s)$ będziemy używać estymacji tej wartości np.:
$\hat{v}_{\pi}(1,2)=\dfrac{G_{\tau_{1}}+G_{\tau_{2}}}{2}$

### Środowisko Food truck


In [1]:
import numpy as np
import gym

In [2]:
# Środowisko rozszerza klasę Env z biblioteki OpenAI Gym
class FoodTruck(gym.Env):
    def __init__(self):
        self.v_demand = [100, 200, 300, 400]
        self.p_demand = [0.3, 0.4, 0.2, 0.1]
        self.capacity = self.v_demand[-1]
        self.days = ['Mon', 'Tue', 'Wed',
                     'Thu', 'Fri', "Weekend"]
        self.unit_cost = 4
        self.net_revenue = 7
        self.action_space = [0, 100, 200, 300, 400]
        # stan środowiska - (dzień tygodnia, zapas burgerów na początku dnia)
        # stan środowiska = obserwacja agenta (środowisko w pełni obserwowalne)
        self.state_space = [("Mon", 0)] \
                           + [(d, i) for d in self.days[1:]
                              for i in [0, 100, 200, 300]]

    # metoda obliczająca następny stan środowiska oraz nagrodę
    def get_next_state_reward(self, state, action, demand):
        day, inventory = state
        result = {}
        result['next_day'] = self.days[self.days.index(day)
                                       + 1]
        result['starting_inventory'] = min(self.capacity, inventory + action)
        result['cost'] = self.unit_cost * action
        result['sales'] = min(result['starting_inventory'], demand)
        result['revenue'] = self.net_revenue * result['sales']
        result['next_inventory'] = result['starting_inventory'] - result['sales']
        result['reward'] = result['revenue'] - result['cost']
        return result

    def get_transition_prob(self, state, action):
        next_s_r_prob = {}
        for ix, demand in enumerate(self.v_demand):
            result = self.get_next_state_reward(state,
                                                action,
                                                demand)
            next_s = (result['next_day'], result['next_inventory'])
            reward = result['reward']
            prob = self.p_demand[ix]
            if (next_s, reward) not in next_s_r_prob:
                next_s_r_prob[next_s, reward] = prob
            else:
                next_s_r_prob[next_s, reward] += prob
        return next_s_r_prob

    # metody potrzebne do symulacji środowiska
    def reset(self):
        self.day = "Mon"
        self.inventory = 0
        state = (self.day, self.inventory)
        return state

    def is_terminal(self, state):
        day, inventory = state
        if day == "Weekend":
            return True
        else:
            return False

    def step(self, action):
        demand = np.random.choice(self.v_demand, p=self.p_demand)
        result = self.get_next_state_reward((self.day, self.inventory),
                                            action,
                                            demand)
        self.day = result['next_day']
        self.inventory = result['next_inventory']
        state = (self.day, self.inventory)
        reward = result['reward']
        done = self.is_terminal(state)
        info = {'demand': demand, 'sales': result['sales']}

        return state, reward, done, info

C:\Users\Basia\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Wybranie akcji dla danej strategii
def choose_action(state, policy):
    prob_a = policy[state]
    action = np.random.choice(a=list(prob_a.keys()), p=list(prob_a.values()))
    return action

In [4]:
# metoda zwracająca przykładową strategię działania
def base_policy(states):
    policy = {}
    for s in states:
        day, inventory = s
        prob_a = {}
        if inventory >= 300:
            prob_a[0] = 1
        else:
            prob_a[200 - inventory] = 0.5
            prob_a[300 - inventory] = 0.5
        policy[s] = prob_a
    return policy  # dict: stan -> {akcja: prawdopodobieństwo}

### "First-visit" vs "every-visit" Monte Carlo

Załóżmy, że chcemy estymować wartość stanu $v_{\pi}(1,2)$. Na rysunku poniżej widzimy trzy trajektorie: $\tau_{1}, \tau_{2}, \tau_{3}, \tau_{4}$. Żaden z nich nie rozpoczyna się w punkcie $(1,2)$, ale mimo to możemy użyć trajektorii $\tau_{1}, \tau_{2}, \tau_{3}$ do estymacji.
Zauważmy, że $\tau_{3}$ dwukrotnie przechodzi przez punkt $(1,2)$. Czy możemy użyć sumy nagród od pierwszej wizyty (first-visit) w tym punkcie, czy od każdej z wizyt (every-visit)?

![](img/first_vs_every.png)

Oba podejścia są poprawne. "Every-visit" jest prostszy w użyciu przez aproksymatory funkcji (jak sieci neuronowe).

In [5]:
def first_visit_return(returns, trajectory, gamma):
    """

    :param returns: dict, klucze są stanami a wartością jest lista sum nagród uzyskanych z różnych trajektorii
    :param trajectory: lista krotek: (stan, akcja, nagroda)
    :param gamma: współczynnik dyskontowania
    :return:
    """
    G = 0
    T = len(trajectory) - 1
    for t, sar in enumerate(reversed(trajectory)):
        s, a, r = sar
        G = r + gamma * G
        first_visit = True
        for j in range(T - t):
            if s == trajectory[j][0]:
                first_visit = False
        if first_visit:
            if s in returns:
                returns[s].append(G) # dodaje sumę nagród z trajektorii do listy
            else:
                returns[s] = [G]
    return returns

In [6]:
def get_trajectory(env, policy):
    trajectory = []
    state = env.reset()
    done = False
    sar = [state]
    while not done:
        action = choose_action(state, policy)
        state, reward, done, info = env.step(action)
        sar.append(action)
        sar.append(reward)
        trajectory.append(sar)
        sar = [state]
    return trajectory

In [7]:
def first_visit_mc(env, policy, gamma, n_trajectories):
    np.random.seed(0)
    returns = {}
    v = {}
    for i in range(n_trajectories):
        trajectory = get_trajectory(env, policy)
        returns = first_visit_return(returns,
                                     trajectory,
                                     gamma)
    for s in env.state_space:
        if s in returns:
            v[s] = np.round(np.mean(returns[s]), 1)
    return v

In [8]:
foodtruck = FoodTruck()
policy = base_policy(foodtruck.state_space)

In [9]:
v_est = first_visit_mc(foodtruck, policy, 1, 10000)
v_est

{('Mon', 0): 2515.9,
 ('Tue', 0): 1959.1,
 ('Tue', 100): 2362.2,
 ('Tue', 200): 2765.2,
 ('Wed', 0): 1411.3,
 ('Wed', 100): 1804.2,
 ('Wed', 200): 2198.9,
 ('Thu', 0): 852.9,
 ('Thu', 100): 1265.4,
 ('Thu', 200): 1644.4,
 ('Fri', 0): 301.1,
 ('Fri', 100): 696.5,
 ('Fri', 200): 1097.2}

In [14]:
def every_visit_return(returns, trajectory, gamma):
    G = 0
    T = len(trajectory) - 1
    for t, sar in enumerate(reversed(trajectory)):
        s, a, r = sar
        G = r + gamma * G
        if s in returns:
            returns[s].append(G) # dodaje sumę nagród z trajektorii do listy
        else:
            returns[s] = [G]
    return returns

In [15]:
def every_visit_mc(env, policy, gamma, n_trajectories):
    np.random.seed(0)
    returns = {}
    v = {}
    for i in range(n_trajectories):
        trajectory = get_trajectory(env, policy)
        returns = every_visit_return(returns,
                                     trajectory,
                                     gamma)
    for s in env.state_space:
        if s in returns:
            v[s] = np.round(np.mean(returns[s]), 1)
    return v

In [16]:
v_est = every_visit_mc(foodtruck, policy, 1, 10000)
v_est

{('Mon', 0): 2515.9,
 ('Tue', 0): 1959.1,
 ('Tue', 100): 2362.2,
 ('Tue', 200): 2765.2,
 ('Wed', 0): 1411.3,
 ('Wed', 100): 1804.2,
 ('Wed', 200): 2198.9,
 ('Thu', 0): 852.9,
 ('Thu', 100): 1265.4,
 ('Thu', 200): 1644.4,
 ('Fri', 0): 301.1,
 ('Fri', 100): 696.5,
 ('Fri', 200): 1097.2}

Widzimy, że estymowane wartości są bliskie wartościom rzeczywistym:

![](img/v_pi_true_values.png)

**Problem**, który się pojawia, to brak wyznaczonych wartości stanów dla stanów, gdzie agent na początku dnia wypełnia zasoby o 300 jednostek. Dzieje się tak, dlatego, że użyta strategia nie pozwala na odwiedzenie tych stanów!

## Monte Carlo control

Problem sterowania MC polega na znalezieniu optymalnej strategii wykorzystując doświadczenie agenta (trajektorie). Aby mieć pewność, że każdy ze stanów zostanie odwiedzony, należy zaimplementować strategię **eksploracji**. W poniższym przykładzie wykorzystamy metodę $\epsilon-greedy$.

##### $\epsilon-greedy$

Metoda ta wybiera losową akcję z prawdopodobieństwem $\epsilon$ oraz akcję maksymalizującą funkcję akcja-wartość z prawdopodobieństwem $1-\epsilon$.

### Off-policy method
Strategia wykorzystująca eksplorację nazywana jest **strategią zachowania (behavior policy), $b$**, natomiast strategia wykorzystująca jedynie najlepsze akcje (maksymalizacja funkcji akcja-wartość) nazywana jest **strategią docelową (target policy), $\pi$**.
Powyższa metoda nazywana jest **off-policy**, ponieważ korzysta z obu wspomnianych strategii. Pierwsza z nich używana jest do generowania danych (trajektorii), natomiast druga jest docelową, optymalną strategią, której szukamy.

![](img/off_policy_mc_control.png)

W metodach **on-policy** strategia zachowania jest używana do generacji danych oraz służy jako szukana strategia.

In [10]:
# Przykład zwracanej wartości: {0: 0.96, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.01}
# eps=0.05, najlepsza akcja=0
def get_eps_greedy(actions, eps, a_best):
    prob_a = {}
    n_a = len(actions)
    for a in actions:
        if a == a_best:
            prob_a[a] = 1 - eps + eps/n_a
        else:
            prob_a[a] = eps/n_a
    return prob_a

In [11]:
import operator

In [12]:
def get_random_policy(states, actions):
    policy = {}
    n_a = len(actions)
    for s in states:
        policy[s] = {a: 1/n_a for a in actions}
    return policy

In [13]:
def off_policy_mc(env, n_iter, eps, gamma):
    np.random.seed(0)
    states =  env.state_space
    actions = env.action_space
    Q = {s: {a: 0 for a in actions} for s in states}
    C = {s: {a: 0 for a in actions} for s in states}
    target_policy = {}
    behavior_policy = get_random_policy(states,
                                        actions)
    for i in range(n_iter):
        if i % 10000 == 0:
            print("Iteration:", i)
        trajectory = get_trajectory(env, behavior_policy)
        G = 0
        W = 1
        for t, sar in enumerate(reversed(trajectory)):
            s, a, r = sar
            G = r + gamma * G
            C[s][a] += W
            Q[s][a] += (W/C[s][a]) * (G - Q[s][a])
            a_best = max(Q[s].items(), key=operator.itemgetter(1))[0]
            target_policy[s] = a_best
            behavior_policy[s] = get_eps_greedy(actions, eps, a_best)
            if a != target_policy[s]:
                break
            W = W / behavior_policy[s][a]
    target_policy = {s: target_policy[s] for s in states
                     if s in target_policy}
    return target_policy, Q

In [37]:
policy, Q = off_policy_mc(foodtruck, 300000, 0.05, 1)
policy

Iteration: 0
Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000
Iteration: 200000
Iteration: 210000
Iteration: 220000
Iteration: 230000
Iteration: 240000
Iteration: 250000
Iteration: 260000
Iteration: 270000
Iteration: 280000
Iteration: 290000


{('Mon', 0): 400,
 ('Tue', 0): 400,
 ('Tue', 100): 300,
 ('Tue', 200): 200,
 ('Tue', 300): 100,
 ('Wed', 0): 400,
 ('Wed', 100): 300,
 ('Wed', 200): 200,
 ('Wed', 300): 100,
 ('Thu', 0): 300,
 ('Thu', 100): 200,
 ('Thu', 200): 100,
 ('Thu', 300): 0,
 ('Fri', 0): 200,
 ('Fri', 100): 100,
 ('Fri', 200): 0,
 ('Fri', 300): 0}

## Zadanie

1. Co należy zmienić w metodzie *first_visit_return*, aby zaimplementować metodę *every-visit*? Wynik $\hat{v}$ powinien być identyczny z przypadkiem *first-visit*.
2. Zaimplementuj metodę $\epsilon-greedy$.
3. Opisz metodę *importance sampling* wykorzystywaną w metodach Monte Carlo. Jaką rolę odgrywa ona w powyższej metodzie *off-policy*.

1. Co należy zmienić w metodzie first_visit_return, aby zaimplementować metodę every-visit? Wynik  𝑣̂ powinien być identyczny z przypadkiem first-visit. <br>
<br>
Usunięcie warunku (pętla for oraz if'y związane z first_visit). Zaimplementowałam metoda every_visit pod metodą first_visit. Wyniki z obu metod są identyczne.

3. Opisz metodę importance sampling wykorzystywaną w metodach Monte Carlo. Jaką rolę odgrywa ona w powyższej metodzie off-policy. <br>
<br>
Metoda importance sampling to ogólna technika szacowania wartości oczekiwanych w jednym rozkładzie na podstawie próbek z innego rozkładu. Stosujemy tą metodę do uczenia się poza strategią (off-policy learning) poprzez ważenie zwrotów według względnego prawdopodobieństwa wystąpienia ich trajektorii w ramach strategii docelowej i strategii zachowania.

## Materiały

- Sutton, Barto, "Reinforcement Learning an Introduction", 2nd Edition, Chapter 5.
- [Monte Carlo And Off-Policy Methods](https://www.youtube.com/watch?v=bpUszPiWM7o)
- Mastering Reinforcement Learning with Python, p. 152-163